In [108]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [109]:
# Plot candle price
pair ="EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [110]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [111]:
df.describe() #show for numerical values

,volume
count,5000.000000
mean,3969.037600
std,2928.376527
min,125.000000
25%,2091.500000
50%,3270.500000
75%,4989.250000
max,27524.000000


In [112]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [113]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [114]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()



In [115]:
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)
df_ma.head(9)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-01-20T05:00:00.000000000Z,1.08293,1.08357,1.08251,1.08352,1.082584,1.081147
1,2023-01-20T06:00:00.000000000Z,1.08352,1.08368,1.08240,1.08247,1.082674,1.081087
2,2023-01-20T07:00:00.000000000Z,1.08247,1.08467,1.08209,1.08402,1.082992,1.081147
3,2023-01-20T08:00:00.000000000Z,1.08403,1.08590,1.08386,1.08408,1.083190,1.081222
4,2023-01-20T09:00:00.000000000Z,1.08406,1.08520,1.08285,1.08310,1.083281,1.081298
5,2023-01-20T10:00:00.000000000Z,1.08310,1.08474,1.08260,1.08388,1.083345,1.081367
6,2023-01-20T11:00:00.000000000Z,1.08388,1.08417,1.08247,1.08250,1.083309,1.081416
7,2023-01-20T12:00:00.000000000Z,1.08252,1.08278,1.08140,1.08185,1.083242,1.081454
8,2023-01-20T13:00:00.000000000Z,1.08182,1.08276,1.08024,1.08066,1.083088,1.081483


In [116]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [117]:
df_ma['DIFF_PREV']= df_ma.DIFF.shift(1)

In [118]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [119]:
df_ma['IS_TRADE']= df_ma.apply(is_trade, axis=1)

In [120]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [121]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
113,2023-01-26T22:00:00.000000000Z,1.08916,1.08916,1.08878,1.08908,1.089071,1.089080,-0.000008,0.000181,True
191,2023-02-01T04:00:00.000000000Z,1.08628,1.08690,1.08627,1.08658,1.086114,1.086069,0.000045,-0.000204,True
238,2023-02-03T03:00:00.000000000Z,1.08896,1.08959,1.08894,1.08941,1.091483,1.091924,-0.000441,0.000308,True
325,2023-02-08T18:00:00.000000000Z,1.07284,1.07394,1.07244,1.07340,1.073667,1.073646,0.000021,-0.000074,True
330,2023-02-08T23:00:00.000000000Z,1.07124,1.07172,1.07100,1.07155,1.073001,1.073066,-0.000066,0.000104,True


In [105]:
df_ma.head(9)

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2023-01-20T06:00:00.000000000Z,1.08352,1.08368,1.08240,1.08247,1.082674,1.081087,0.001586,NaN,False
1,2023-01-20T07:00:00.000000000Z,1.08247,1.08467,1.08209,1.08402,1.082992,1.081147,0.001845,0.001586,False
2,2023-01-20T08:00:00.000000000Z,1.08403,1.08590,1.08386,1.08408,1.083190,1.081222,0.001968,0.001845,False
3,2023-01-20T09:00:00.000000000Z,1.08406,1.08520,1.08285,1.08310,1.083281,1.081298,0.001983,0.001968,False
4,2023-01-20T10:00:00.000000000Z,1.08310,1.08474,1.08260,1.08388,1.083345,1.081367,0.001978,0.001983,False
5,2023-01-20T11:00:00.000000000Z,1.08388,1.08417,1.08247,1.08250,1.083309,1.081416,0.001893,0.001978,False
6,2023-01-20T12:00:00.000000000Z,1.08252,1.08278,1.08140,1.08185,1.083242,1.081454,0.001787,0.001893,False
7,2023-01-20T13:00:00.000000000Z,1.08182,1.08276,1.08024,1.08066,1.083088,1.081483,0.001605,0.001787,False
8,2023-01-20T14:00:00.000000000Z,1.08066,1.08260,1.08046,1.08220,1.083030,1.081531,0.001499,0.001605,False


In [122]:
df_plot = df_ma.iloc[80:120]

In [123]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))


fig.update_layout(width=1000,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()
